In [ ]:
# PLOT Plank Blackbody curve including subinterval version 2.0

#%matplotlib inline
#%matplotlib widget
#%matplotlib notebook

from ipywidgets import *
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import csv

#base directory
bdir="C:\\Users\Owner\Documents\JWD\Calculators\Tools\BAFDR"

# Load Modtran data
modinputdata = bdir + r"\ModtranDataTransposed.csv"
modinputdesc = bdir + r"\Modtrandesc.csv"
#print(modinputdata)

MTD = np.loadtxt(modinputdata, delimiter=',')
with open(modinputdesc, 'r') as file:
    MTdesc = file.readlines()

# Load BAFDR-M inputs
desc_file_path = bdir + r'\BAFDR_M_inputs.csv'

with open(desc_file_path, 'r') as file:
    # Read lines and split into variables using commas as separator
    lines = [line.strip().split(',') for line in file.readlines()]

# Extract header and data separately
header = lines[0]  # Assuming the header is in the first row
data_rows = lines[1:]

# Convert the header to a list of strings
input_header = header  
    
# Convert the list of lists to a 2D NumPy array
#inputs_array = np.array([list(map(float, data_rows))])

# Convert the list of lists (excluding the header) to a 2D NumPy array
inputs_array = np.array([list(map(float, row)) for row in data_rows])

# DEBUG ONLY Print the shape and values of the array
#print("Shape of the array:", inputs_array.shape)
#print("Values of the array:")
#print(inputs_array)

#OUTPUT prep Section
# Output file path
case_i=0 # progress counter just FYI
output_file_path = bdir + r'\BAFDR_M_output.csv'
column_header = [" Radiance", "Flux", "Isig", "Jsig", "Idark", "Jdark", "TBLIP", "MaxBLIP", "Well Cap", "Well Fill", "ReadNoise", "Limiting Noise", "MaxSNR", "DynRange", "SNR", "NEC", "NEI", "DSTAR", "NEP", "NETD"]
with open(output_file_path, 'w',newline='') as output_file:
    csv_writer = csv.writer(output_file)
    
    # Writing the headers to the file
    csv_writer.writerow(column_header)

    #start calculations
    for row in inputs_array:

            mtnum, emit, Temp, wvl1, wvl2, f, D, p, tran, DWL1, DWL2, ScaleFactor, QE, OperT, Vn_Amp_Ampin, Vmax, Res_ADC, CifF, Tint = map(float, row)
            mtnum = int(mtnum)
            Res_ADC = int(Res_ADC)

            # Wavelength lower limit, upper limit, and step size
            # These must match the modtran file
            lowl, upl, step = 1.0, 15.025, 0.025
            pi = np.pi

            # Step out wavelengths, match array size of L to wvl
            wvl = np.arange(lowl, upl, step)
            # Set arrays equal in size to wvl
            L, LW = wvl * 3, wvl * 4
            MLsubband, MLsubbandW = 0.0, 0.0

            # Compute radiant intensity values L and sub-band total
            for count, i in enumerate(wvl):
                LW[count] = 37400 / (wvl[count]**5) * (1 / ((2.71828)**(14400 / (wvl[count] * Temp)) - 1))
                L[count] = (1.88e27 / (wvl[count]**4)) * (1 / (np.exp(14388 / (wvl[count] * Temp)) - 1)) / 10000
                if wvl1 < wvl[count] < wvl2:
                    if mtnum == -1:
                        MLsubband += L[count] * step
                        MLsubbandW += LW[count] * step
                    else:
                        MLsubband += L[count] * step * MTD[mtnum, count]
                        MLsubbandW += LW[count] * step * MTD[mtnum, count]

            # Sum of radiances
            fullrad, fullradW = sum(L) * step, sum(LW) * step

            # Radiance times atmospheric transmission
            if mtnum == -1:
                ML, MLW = L, LW
                MTdesc[-1] = "Perfect Transmission"
            else:
                ML, MLW = L * MTD[mtnum], LW * MTD[mtnum]

            # Factor in emittance
            MLe, MLWe = ML * emit, MLW * emit

            # Do for sub band
            MLsubbande, MLsubbandWe = emit * MLsubband, emit * MLsubbandW

            # Convert to radiance
            MLsubbandes, MLsubbandWes = emit * MLsubband / pi, emit * MLsubbandW / pi

            # Find peak L and index of peak
            mrx, mrxw = max(L), max(LW)
            inpeak, inpeakw = np.argmax(L), np.argmax(LW)

            # FLUX Section ******************************************************************
            # Compute Flux on detector
            def F_FLUX (tran, MLsubbandes,p,D,f):

                #Compute Flux on Detector
                Flux = (tran * MLsubbandes * pi * (p * 1e-04)**2 * (D * .01)**2) / (4 * (f * 0.01)**2)
                return Flux

            Flux = F_FLUX (tran, MLsubbandes, p, D, f)
            #print("Flux on detector =", Flux, "photons/sec")


            # Detector Section ******************************************************************
            # Compute Signal and Dark Currents
            # Constants
            q = 1.602e-19 # [C], Elementary charge
            kB = 1.38e-23 # [J/K], Boltzmann's constant

            # Empirical parameters from Rule 07-Revisited: Journal of ELECTRONIC MATERIALS, Vol. 39, No. 7, 2010

            J0 = 8367.00001853855 # [A/cm2]
            Pwr = 0.544071281108481
            C = -1.16239134096245
            WLscale = 0.200847413564122 #[um] 
            WLthreshold = 4.63513642316149 # [um]

            #Computations
            Isig=Flux*QE*q
            Jsig=Isig/(p*1e-4)**2

            if DWL2 > WLthreshold :
                WLe = DWL2
            else : 
                tcalc = WLscale/DWL2-WLscale/WLthreshold
                tcalc2 = np.exp(Pwr*np.log(tcalc))
                WLe=DWL2/(1-tcalc2)
                print ('Equiv Wvl Invoked wvl = ',WLe)

            #print(ScaleFactor,J0,C,q,kB,WLe,OperT)
            Jdark=ScaleFactor*J0*np.exp(C*1.24*q/(kB*WLe*OperT))
            Idark=Jdark*(p*1e-4)**2

            TBLIP=C*1.24*q/(kB*DWL2*np.log(Jsig/(J0*ScaleFactor)))
            MaxBLIP=C*1.24*q/(kB*DWL2*np.log(Jsig/(J0*ScaleFactor)/(QE)))
            #print ("  Isig=",Isig,"  Jsig=",Jsig," Idark=", Idark, "  JDark=", Jdark)
            #print ("TBLIP = ", TBLIP, "MaxBLIP=", MaxBLIP)


            #ROIC Section **************************************************************************
            #BEGIN ROIC CALCULATIONS

            # Calculation vector parameters for graphing
            t = np.logspace(-9,-1,1000) # [s], Integration time vector
            t1 = np.ones(t.shape) # Ones vector of size(t)
            Rdata= 5000
            Npix = 1

            #Ci = CifF * 1e-15 # [F], Integration capacitance (get value in F  why not 1e-16?)
            Ci = CifF * 1e-16 # [F], Integration capacitance (get value in F)
            Vn_Amp_Ampin = Vn_Amp_Ampin * t1 # [Vrms], Amplifier noise referred to amplifier input (multiply by ones vector)

            # Total pixel current
            Itot = Isig + Idark # [A]

            # ADC conversion gain
            G_ADC = 2**Res_ADC / Vmax # [DU/V], ADC gain

            # Quantization noise
            #Dn_quant_out = 1/np.sqrt(12) * t1 # [DUrms]  why is this 12???
            Dn_quant_out = 1/np.sqrt(Res_ADC) * t1 # [DUrms]

            # Integrated pixel voltage
            Vtot_pix = np.minimum(Itot * t /Ci,Vmax) # [V], Total
            Vsig_pix = Vtot_pix * Isig/Itot # [V], Signal only

            # Pixel noise sources
            Vn_shot_pix = np.sqrt(q * Vtot_pix /Ci) * t1 # [Vrms], Shot noise
            Vn_kTC_pix = np.sqrt(kB * OperT /Ci) * t1 # [Vrms], kTC (reset) noise

            # Propagate signal to output
            Dsig_out = Vsig_pix * G_ADC # [DU]

            # Propagate all noise sources to output
            Dn_shot_out = Vn_shot_pix * G_ADC # [DUrms]
            Dn_kTC_out = Vn_kTC_pix * G_ADC # [DUrms]
            Dn_Amp_out = Vn_Amp_Ampin * G_ADC # [DUrms]

            # Sum all noises at output
            Dntot_out = np.sqrt(Dn_shot_out**2 + Dn_kTC_out**2 + Dn_Amp_out**2 + Dn_quant_out**2) # [DUrms]

            # Compute SNR
            SNR = 20*np.log10(Dsig_out/Dntot_out) # [dB]


            ## Compute other results

            # Well capacity and well fill
            Nwell = Ci * Vmax / q # [#e-]
            wf = str(round(100*(Tint * Itot)/(q*Nwell),2))

            # Read noise in electrons
            Nnread = Dntot_out[0] / G_ADC * Ci / q # [#e-]

            # Max SNR
            SNRmax = np.amax(SNR) # [dB]

            # Dynamic range
            DR = 20*np.log10(np.amax(Dsig_out)/Dntot_out[0]) # [dB]

            # Saturation time
            tSat = Nwell * q / Itot # [s]  #plus fixed noise??  

            i=0
            index = 77777
            for i in range (0, 1000):
                if t[i] < Tint:
                    i+=1
                else:
                    index=i
                    break

            ## Define limiting noise string

            # Determine limiting noise
            DnmaxIndex = np.argmax((Dn_kTC_out[0],Dn_Amp_out[0],Dn_quant_out[0],Dn_shot_out[i])) 

            # Define string
            if DnmaxIndex == 0:
                LimitingNoise = 'reset'
            elif DnmaxIndex == 1:
                LimitingNoise = 'amplifier'
            elif DnmaxIndex == 2:
                LimitingNoise = 'quantization' 
            elif DnmaxIndex == 3:
                LimitingNoise = 'SHOT'


            #Radiometery Calculations section ****************************************************

            #constants
            c=2.9979e8 #m/sec
            c1=3.7418e4 #watt-um^4/cm^2
            c2=1.4388e4 #um-K
            h=6.6e-34 # m2 kg / s  plancks constant
            q=1.6e-19 #coulomg charge of e

            #planck derivative 
            wvl = ((wvl2-wvl1)/2 + wvl1)
            wvlm = wvl*1e-6
            t1 = np.exp(c2/(wvl*Temp))  #unitless - temporary constant for eqn below
            #print((wvl*Temp),c2,(c2/(wvl*Temp)),'t1=',t1)
            t2= c1/wvl**5
            t3= (1/(t1 - 1))
            M = t2*t3 # Watt/(cm^2 - um) M 5.11  pg126
            L=M/np.pi
            PLDR2 = (L*c2*t1)/(wvl*(Temp**2)*(t1 - 1)) #planck function deriviative Holst yello textbook 3.46 pg55 
            #print('Temp=',Temp,'wvl=',wvl,'L= ',L, '  PLDR= ', PLDR2)

            #Caclulate number of Photons, Photoelectroncs, Noise electrons and SNR
            NPe = Isig*Tint/q #Photo electrons
            NPhoton = NPe/QE #back out incident photons
            PNe = np.sqrt((NPhoton))  # Photon Shot Noise
            DNe = np.sqrt((Idark*Tint)/q)  # Dark Current Noise
            TNe = np.sqrt((DNe**2 + PNe**2 + Nnread**2)) #add noise in quadrature - Dark Noise e + photon/flux noise e + ROIC fixed noise floor
            NEC = TNe # Noise Equiv Carriers = Total Noise Electrons 
            NEI = NEC/(QE*Tint*(p*1e-4)**2)
            SNR2 = NPhoton/NEC  #Nphoto-electrons over total noise electrons
            SNR2L = 20*np.log10(SNR2)
            #print(Isig,Idark,SNR,NEC)

            #DSTAR calc
            NEP = NEI*(h*c/wvlm)*((p*1e-4)**2)  #Noise Equiv Power = NEI(hc/wvlm)(Area-det) 
            NEB = 1/(2*Tint) #Noise Equiv Bandwidth
            DSTAR = ((p*1e-4)* np.sqrt(NEB))/NEP  # cm-Hz^1/2 per Watt  D* eqn 8.24 2nd edition

            #NETD calc
            num =((f/D)**2)*np.sqrt(1/(2*Tint))   #numerator
            den = tran*p*1e-4*DSTAR*PLDR2  #denominator
            NETD = (4*num)/(np.pi*den)  #NETD formula 14.17 in 2nd edition


            output_row=[MLsubbandes , Flux , Isig , Jsig , Idark , Jdark , TBLIP , MaxBLIP , Nwell , \
            wf , Nnread , LimitingNoise , round(SNRmax,2) , round(DR,2) , round(SNR[index],2) , NEC , NEI , DSTAR , NEP , NETD ]

            #print(output_row)
            csv_writer.writerow(output_row)
            print("Completed run:", case_i+1)
            case_i+=1
print("Done")
    